In [1]:
%reset
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.stats
import mne
from mne.time_frequency import tfr_morlet
from mne.stats import permutation_cluster_1samp_test
import gc
import os
import copy
import pickle
from os.path import exists
import mne
import numpy as np
from mne import create_info
from IPython.utils import io
import yasa
import seaborn as sns
import pandas as pd
import matlab
import matlab.engine
import os
import shutil
from scipy.signal import butter, lfilter
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from scipy.signal import freqz
from scipy import signal

def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def overlap(a, b):
    return a[1] >= b[0] and a[0] <= b[1]

gc.collect()

30233

In [2]:
pkl_dir = "C:\\Users\\User\\Cloud-Drive\\BigFiles\\OmissionExpOutput\\\imported_eventDetectionChan\\filter0.35"
import_type = "eventDetectionChan"
output_dir_name = 'eventDetection\\done_on_imported'
import_path = f'{pkl_dir}\\{import_type}.pkl'

electrode_column_name = 'electrode'
header = np.array(['Onset',"Duration","Annotation"])

Batch Nir&Andrillon Spindle Detection

In [3]:
def detect_sw_AndrillonNir(file_ids,output_key,electrodes_names):
    andriNir_code_dir = os.getcwd()
    andriNir_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}\\Andrillon_Nir"
    andriNir_aux_output_dir = f"{andriNir_output_dir}\\aux_mats"

    if os.path.exists(andriNir_output_dir):
        shutil.rmtree(andriNir_output_dir)
    os.mkdir(andriNir_output_dir)
    os.mkdir(andriNir_aux_output_dir)

    sw_output_columns = ['wavest' ,'wavend', 'mdpt', 'poszx', 'period/SR' ,'abs(b)', 'bx', 'c' ,'cx' ,'maxb2c', 'n1', 'n1x', 'nEnd', 'nEndx' ,'p1', 'p1x' ,'meanAmp', 'nump', 'nperiod/SR', 'p2p', 'mxdn', 'mxup' ,'currentStage']

    for id in file_ids:
        datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
        datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
        scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'], verbose=True)
        sw_allElectrodes = pd.DataFrame()

        for electd_i, electrode_name_eventDetect in enumerate(electrodes_names):
            curr_electrode_num = np.where(configu['electrodes_names'] == electrode_name_eventDetect)[0][0]

            ## create aux files to use in MATLAB
            datafile_1elect_eeg = datafile_data[curr_electrode_num,:]
            if 2 not in datafile_scoring or 3 not in datafile_scoring:
                continue
            
            mat_to_save = {'datafile_data': datafile_1elect_eeg, 'scoring_upsampled': scoring_upsampled, 'sample_freq': configu['sample_freq'], 'electrode_name':electrode_name_eventDetect}
            scipy.io.savemat(f"{andriNir_aux_output_dir}\\{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_{electrode_name_eventDetect}AndScoring.mat",mat_to_save)

            ## run Andrillon & Nir SW detection over all subjects files
            eng = matlab.engine.start_matlab()
            eng.cd(andriNir_code_dir, nargout=0)
            out = eng.batch_useAndrillonNirSWDetection(andriNir_aux_output_dir, andriNir_output_dir,nargout=0)
            eng.quit()

            ## add the SW data to the main subject dictionary
            try: 
                sw_file_name = f"{andriNir_output_dir}\\{output_key}_{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_{electrode_name_eventDetect}AndScoring.mat"
                matlabImport = scipy.io.loadmat(sw_file_name, simplify_cells=True)
            except Exception: 
                print(f"Error importing spindles sub file at: {allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}")
                continue

            sw = matlabImport['allWaves']
            if np.size(np.shape(sw)) == 1: 
                temp = np.zeros((1,np.size(sw)),dtype=object)
                temp[0] = sw
                sw = temp
            if np.size(sw) >0 :
                df = pd.DataFrame(np.double(sw))
                df.columns = sw_output_columns
                tile_electrode = np.tile(f"{electrode_name_eventDetect}", sw.shape[0])[None].T
                df[electrode_column_name] = tile_electrode
                
                allsubsdata_perFile[id][f"{output_key}@@{electrode_name_eventDetect}"] = df
                if electd_i ==0: sw_allElectrodes =  df
                else:  sw_allElectrodes = pd.concat([sw_allElectrodes,df])
    
        if not sw_allElectrodes.empty:
            allsubsdata_perFile[id][output_key]  = sw_allElectrodes
    shutil.rmtree(andriNir_aux_output_dir)
def detect_kComp_AndrillonNir(file_ids,output_key,electrodes_names):
    andriNir_code_dir = os.getcwd()
    andriNir_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}\\Andrillon_Nir"
    andriNir_aux_output_dir = f"{andriNir_output_dir}\\aux_mats"

    if os.path.exists(andriNir_output_dir):
        shutil.rmtree(andriNir_output_dir)
    os.mkdir(andriNir_output_dir)
    os.mkdir(andriNir_aux_output_dir)

    kcomp_output_columns = ['wavest' ,'wavend', 'mdpt', 'poszx', 'period' ,'abs(b)', 'bx', 'c' ,'cx' ,'maxb2c', 'n1', 'n1x', 'nEnd', 'nEndx' ,'p1', 'p1x' ,'meanAmp', 'nump', 'nperiod/SR', 'p2p', 'mxdn', 'mxup' ,'currentStage']

    for id in file_ids:
        datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
        datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
        scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'], verbose=True)
        kcomp_allElectrodes = pd.DataFrame()

        for electd_i, electrode_name_eventDetect in enumerate(electrodes_names):
            curr_electrode_num = np.where(configu['electrodes_names'] == electrode_name_eventDetect)[0][0]

            ## create aux files to use in MATLAB
            datafile_1elect_eeg = datafile_data[curr_electrode_num,:]
            if 2 not in datafile_scoring or 3 not in datafile_scoring:
                continue
            
            mat_to_save = {'datafile_data': datafile_1elect_eeg, 'scoring_upsampled': scoring_upsampled, 'sample_freq': configu['sample_freq'], 'electrode_name':electrode_name_eventDetect}
            scipy.io.savemat(f"{andriNir_aux_output_dir}\\{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_{electrode_name_eventDetect}AndScoring.mat",mat_to_save)

            ## run Andrillon & Nir Kcomp detection over all subjects files
            eng = matlab.engine.start_matlab()
            eng.cd(andriNir_code_dir, nargout=0)
            out = eng.batch_useAndrillonNirKcompDetection(andriNir_aux_output_dir, andriNir_output_dir,nargout=0)
            eng.quit()

            ## add the SW data to the main subject dictionary
            try: 
                kcomp_file_name = f"{andriNir_output_dir}\\{output_key}_{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_{electrode_name_eventDetect}AndScoring.mat"
                matlabImport = scipy.io.loadmat(kcomp_file_name, simplify_cells=True)
            except Exception: 
                print(f"Error importing kcomp sub file at: {kcomp_file_name}")
                continue

            kComps = matlabImport['kcomps']
            if np.size(np.shape(kComps)) == 1: 
                temp = np.zeros((1,np.size(kComps)),dtype=object)
                temp[0] = kComps
                kComps = temp
            if np.size(kComps) >0 :
                df = pd.DataFrame(np.double(kComps))
                df.columns = kcomp_output_columns
                tile_electrode = np.tile(f"{electrode_name_eventDetect}", kComps.shape[0])[None].T
                df[electrode_column_name] = tile_electrode
                
                allsubsdata_perFile[id][f"{output_key}@@{electrode_name_eventDetect}"] = df
                if electd_i ==0: kcomp_allElectrodes =  df
                else:  kcomp_allElectrodes = pd.concat([kcomp_allElectrodes,df])
    
        if not kcomp_allElectrodes.empty:
            allsubsdata_perFile[id][output_key]  = kcomp_allElectrodes

        
    shutil.rmtree(andriNir_aux_output_dir)
def detect_ss_AndrillonNir(file_ids,minMax_sd_ver,output_key,electrodes_names):
    andriNir_code_dir = os.getcwd()
    andriNir_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}\\Andrillon_Nir"
    andriNir_aux_output_dir = f"{andriNir_output_dir}\\aux_mats"

    if os.path.exists(andriNir_output_dir):
        shutil.rmtree(andriNir_output_dir)
    os.mkdir(andriNir_output_dir)
    os.mkdir(andriNir_aux_output_dir)

    spindles_output_columns =  ['spindleStartTime', 'spindleEndTime', 'peakTime', 'peakEnergy', 'peakEnergyNorm', 'freqSpindle', 'spindleDuration/SR', 'PowerSP', 'PowerAlpha', 'currentStage']

    for id in file_ids:
        datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
        datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
        scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'], verbose=True)
        ss_allElectrodes = pd.DataFrame()

        for electd_i, electrode_name_eventDetect in enumerate(electrodes_names):
            curr_electrode_num = np.where(configu['electrodes_names'] == electrode_name_eventDetect)[0][0]

            ## create aux files to use in MATLAB
            datafile_1elect_eeg = datafile_data[curr_electrode_num,:]
            if 2 not in datafile_scoring or 3 not in datafile_scoring:
                continue
            
            DetectionThreshold = minMax_sd_ver[0]
            RejectThreshold = minMax_sd_ver[1]
            StartEndThreshold = minMax_sd_ver[2]
            mat_to_save = {'datafile_data': datafile_1elect_eeg, 'scoring_upsampled': scoring_upsampled, 'sample_freq': configu['sample_freq'], 'electrode_name':electrode_name_eventDetect, 'DetectionThreshold':DetectionThreshold,'RejectThreshold':RejectThreshold, 'StartEndThreshold':StartEndThreshold}
            scipy.io.savemat(f"{andriNir_aux_output_dir}\\{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_{electrode_name_eventDetect}AndScoring.mat",mat_to_save)

            ## run Andrillon & Nir SS detection over all subjects files
            eng = matlab.engine.start_matlab()
            eng.cd(andriNir_code_dir, nargout=0)
            out = eng.batch_useAndrillonNirSSDetection(andriNir_aux_output_dir, andriNir_output_dir,nargout=0)
            eng.quit()

            ## add the spindles data to the main subject dictionary
            try: 
                spindles_file_name = f"{andriNir_output_dir}\\SS_AN_{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_{electrode_name_eventDetect}AndScoring.mat"
                matlabImport = scipy.io.loadmat(spindles_file_name, simplify_cells=True)
            except Exception: 
                print(f"Error importing spindles sub file at: {allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}")
                continue

            ss = matlabImport['spindles']
            if np.size(np.shape(ss)) == 1: 
                temp = np.zeros((1,np.size(ss)),dtype=object)
                temp[0] = ss
                ss = temp
            if np.size(ss) >0 :
                df = pd.DataFrame(np.double(ss))
                df.columns = spindles_output_columns
                tile_electrode = np.tile(f"{electrode_name_eventDetect}", ss.shape[0])[None].T
                df[electrode_column_name] = tile_electrode
                
                allsubsdata_perFile[id][f'{output_key}@@{electrode_name_eventDetect}'] = df
                if electd_i ==0: ss_allElectrodes =  df
                else:  ss_allElectrodes = pd.concat([ss_allElectrodes,df])
    
        if not ss_allElectrodes.empty:
            allsubsdata_perFile[id][output_key]  = ss_allElectrodes

    minmax_sd_name =  matlabImport['minMax_SD_threshold']
    shutil.rmtree(andriNir_aux_output_dir)
    return minmax_sd_name


def filter_sleep_stages(file_ids,event_key_to_use,after_sleepStage_exclution_key, sleepstages):
    for id in file_ids:
        if event_key_to_use not in allsubsdata_perFile[id]: continue
        event_allElectrodes = allsubsdata_perFile[id][event_key_to_use]
        
        event_filtered_sleepstages = pd.DataFrame([], columns=event_allElectrodes.columns)
        for sleepstage in sleepstages:
            event_filtered_sleepstages = pd.concat([event_filtered_sleepstages , event_allElectrodes.loc[(event_allElectrodes['currentStage'] == sleepstage)]])

        if len(event_filtered_sleepstages) == 0:  print(f"sub:{id} - no events in sleepstages{sleepstages}")
        else: print(f"sub:{id}, before exclude sleep stages:{len(event_allElectrodes)}, after:{len(event_filtered_sleepstages)}")
        
        allsubsdata_perFile[id][after_sleepStage_exclution_key] = event_filtered_sleepstages
def filter_kcompCriteria(file_ids,event_key_to_use,afterkcompCriteria_exclution_key):
            # % Result Matrix
        # %1:  wave beginning (sample)
        # %2:  wave end (sample)
        # %3:  wave middle point (sample)
        # %4:  wave negative half-way (sample)
        # %5:  period in seconds
        # %6:  positive amplitude peak
        # %7:  positive amplitude peak position (sample)
        # %8:  negative amplitude peak
        # %9:  negative amplitude peak position (sample)
        # %10: peak-to-peak amplitude
        # %11: 1st pos peak amplitude
        # %12: 1st pos peak amplitude position (sample)
        # %13: Last pos peak amplitude
        # %14: Last pos peak amplitude position (sample)
        # %15: 1st neg peak amplitude
        # %16: 1st neg peak amplitude position (sample)
        # %17: mean wave amplitude
        # %18: number of positive peaks
        # %19: wave negative half-way period
        # %20: 1st peak to last peak period
        # %21: determines instantaneous positive 1st segement slope on smoothed signal
        # %22: determines maximal negative slope for 2nd segement
        # %23: stage (if scored data)
    for id in file_ids:
        if event_key_to_use not in allsubsdata_perFile[id]: continue
        event_allElectrodes = allsubsdata_perFile[id][event_key_to_use]
        event_filtered_kcompCrit = pd.DataFrame([], columns=event_allElectrodes.columns)

        event_filtered_kcompCrit = event_allElectrodes[
                                                        (event_allElectrodes['period'] >= 0.5) &
                                                        (event_allElectrodes['n1'] < 45) &
                                                        ((event_allElectrodes['currentStage'] == 2.0) | (event_allElectrodes['currentStage'] == 3.0))
                                                       ]

        if len(event_filtered_kcompCrit) == 0:  print(f"sub:{id} - no kcomp with criteria")
        else: print(f"sub:{id}, before wave criteria sleep stages:{len(event_allElectrodes)}, after:{len(event_filtered_kcompCrit)}")
        
        allsubsdata_perFile[id][afterkcompCriteria_exclution_key] = event_filtered_kcompCrit
def filter_waveCriteria(file_ids,event_key_to_use,after_sleepStage_exclution_key):
        # % Result Matrix
        # %1:  wave beginning (sample)
        # %2:  wave end (sample)
        # %3:  wave middle point (sample)
        # %4:  wave negative half-way (sample)
        # %5:  period in seconds
        # %6:  positive amplitude peak
        # %7:  positive amplitude peak position (sample)
        # %8:  negative amplitude peak
        # %9:  negative amplitude peak position (sample)
        # %10: peak-to-peak amplitude
        # %11: 1st pos peak amplitude
        # %12: 1st pos peak amplitude position (sample)
        # %13: Last pos peak amplitude
        # %14: Last pos peak amplitude position (sample)
        # %15: 1st neg peak amplitude
        # %16: 1st neg peak amplitude position (sample)
        # %17: mean wave amplitude
        # %18: number of positive peaks
        # %19: wave negative half-way period
        # %20: 1st peak to last peak period
        # %21: determines instantaneous positive 1st segement slope on smoothed signal
        # %22: determines maximal negative slope for 2nd segement
        # %23: stage (if scored data)
    for id in file_ids:
        if event_key_to_use not in allsubsdata_perFile[id]: continue
        event_allElectrodes = allsubsdata_perFile[id][event_key_to_use]
        event_filtered_waveCrit = pd.DataFrame([], columns=event_allElectrodes.columns)

        event_filtered_waveCrit = event_allElectrodes[(event_allElectrodes['maxb2c'] > 75)]

        if len(event_filtered_waveCrit) == 0:  print(f"sub:{id} - no sw with criteria")
        else: print(f"sub:{id}, before wave criteria sleep stages:{len(event_allElectrodes)}, after:{len(event_filtered_waveCrit)}")
        
        allsubsdata_perFile[id][after_sleepStage_exclution_key] = event_filtered_waveCrit
def group_spindles(ss_key_to_use, uniqeElctds_ss_key, uniqeElctd_ss_key):
    ## create a spindle array, where each spindle occour once and attribued to the electrode where the spindle was the most powerful
    for id in allsubsdata_perFile:
        if ss_key_to_use not in allsubsdata_perFile[id]:  continue
        filterd_events_allElectrodes = allsubsdata_perFile[id][ss_key_to_use].copy(deep=True)
        
        filterd_events_allElectrodes.sort_values(by=['spindleStartTime'],inplace=True) 
        filterd_events_allElectrodes.reset_index(drop=True, inplace=True)
        deleted = filterd_events_allElectrodes.copy(deep=True)
        filtered = pd.DataFrame([], columns = deleted.columns)
        filtered_moreThan1 = pd.DataFrame([], columns = deleted.columns)
        simultan = pd.DataFrame([], columns = deleted.columns)

        while len(deleted)>0:
            if len(simultan)==0:
                simultan = pd.concat([simultan, deleted.iloc[[0]]])
                deleted.drop(deleted.index[0], axis=0, inplace=True)
                still_overlap = True
            else:
                while still_overlap == True and len(deleted)>0:
                    still_overlap = False
                    simultan.reset_index(drop=True, inplace=True) # make sure indexes pair with number of rows

                    ## check now_overlap_all_in_simultan:
                    for index, simultan_row in simultan.iterrows():
                        simultan_0 = simultan_row['spindleStartTime']
                        simultan_1 = simultan_row['spindleEndTime']
                        deleted_0 = deleted.iloc[0]['spindleStartTime']
                        deleted_1 = deleted.iloc[0]['spindleEndTime']
                        if overlap([simultan_0,simultan_1],[deleted_0,deleted_1]):
                            simultan = pd.concat([simultan, deleted.iloc[[0]]])
                            deleted.drop(deleted.index[0], axis=0, inplace=True)
                            still_overlap = True
                            break
                    if still_overlap: continue
                    else:
                        ## check max ps and add to filt
                        # if len(simultan)>1:
                        #     print('hu')
                        simultan.sort_values(by=['PowerSP'],ascending=False,inplace=True)
                        row_df = simultan.iloc[[0]]
                        row_df.iat[0, row_df.columns.get_loc(electrode_column_name)] = np.array(simultan[electrode_column_name])
                        filtered = pd.concat([filtered,row_df])
                        if len(simultan) > 1: filtered_moreThan1 = pd.concat([filtered_moreThan1,row_df])
                        simultan = pd.DataFrame([], columns = deleted.columns)                    
            
        filtered.reset_index(drop=True, inplace=True) # make sure indexes pair with number of rows
        filtered_moreThan1.reset_index(drop=True, inplace=True) # make sure indexes pair with number of rows
        allsubsdata_perFile[id][uniqeElctd_ss_key] = filtered   
        allsubsdata_perFile[id][uniqeElctds_ss_key] = filtered_moreThan1   
        print(f"sub:{id}, before filt:{np.shape(allsubsdata_perFile[id][ss_key_to_use])[0]}, after:{np.shape(allsubsdata_perFile[id][uniqeElctd_ss_key])[0]}, after>1: after:{np.shape(allsubsdata_perFile[id][uniqeElctds_ss_key])[0]}")

def edfViewFormat_scoring_dict(score):
    if score == 0:
        return 'W'
    elif score ==1:
        return 'N1'
    elif score ==2:
        return 'N2'
    elif score ==3:
        return 'N3'
    elif score ==4:
        return 'TREM'
    elif score ==5:
        return 'PREM'
    elif score ==6:
        return 'MOVE'
    elif score ==7:
        return 'ARTIFACT'
    else:
        Exception('no such score')
def add_edfViewFormat_scoring(key_edfScoringFormat):
    for id in allsubsdata_perFile:
        curr_file_scoring = allsubsdata_perFile[id]['scoring']
        new_format_score = np.zeros((len(curr_file_scoring),3), dtype=object)
        for ind, score in enumerate(curr_file_scoring):
                new_format_score[ind,:] = [30*ind,30,edfViewFormat_scoring_dict(score)] ## onset (sec), duration, desc

        allsubsdata_perFile[id][key_edfScoringFormat] = new_format_score
def add_edfViewFormat_ss(file_ids,event_key_for_save, SS_efdViewFormat_key):
        for id in file_ids:
            if event_key_for_save not in allsubsdata_perFile[id]:
                    #print(f"no {event_key_for_save} for sub {id}")
                    continue
            ss_df = allsubsdata_perFile[id][event_key_for_save]
            startTime_arr = np.array(ss_df['spindleStartTime']) / np.double(configu['sample_freq'])
            endTime_arr = np.array(ss_df['spindleEndTime']) / np.double(configu['sample_freq'])
            duration_arr = endTime_arr - startTime_arr
            electd_arr_per_ss = np.array(ss_df[electrode_column_name])
            desc = [f"SS@@{electd_arr[0]}" for electd_arr in electd_arr_per_ss] ## NOTICE: this will work only if the electrode column consists of values which are arrays
            new_format_ss = np.array([startTime_arr,duration_arr,desc]).T ## onset (sec), duration,desc
            allsubsdata_perFile[id][SS_efdViewFormat_key] = new_format_ss
def add_edfViewFormat_sw(file_ids,event_key_for_save, SW_efdViewFormat_key):
        for id in file_ids:
            if event_key_for_save not in allsubsdata_perFile[id]:
                    #print(f"no {event_key_for_save} for sub {id}")
                    continue
            sw_df = allsubsdata_perFile[id][event_key_for_save]
    
            startTime_arr = np.array(sw_df['wavest'])
            endTime_arr = np.array(sw_df['wavend'])
            duration_arr = (endTime_arr - startTime_arr)  / np.double(configu['sample_freq'])
            electd_arr_per_sw = np.array(sw_df[electrode_column_name])
            desc = [f"SW@@{electd_arr}" for electd_arr in electd_arr_per_sw]
            new_format_sw = np.array([startTime_arr / np.double(configu['sample_freq']) ,duration_arr,desc]).T ## onset (sec), duration,desc

            allsubsdata_perFile[id][SW_efdViewFormat_key] = new_format_sw
def add_edfViewFormat_kcomp(file_ids,event_key_for_save, kcomp_efdViewFormat_key):
    for id in file_ids:
        if event_key_for_save not in allsubsdata_perFile[id]:
                #print(f"no {event_key_for_save} for sub {id}")
                continue
        kcomp_df = allsubsdata_perFile[id][event_key_for_save]

        startTime_arr = np.array(kcomp_df['wavest'])
        endTime_arr = np.array(kcomp_df['wavend'])
        duration_arr = np.round(endTime_arr - startTime_arr,2)
        electd_arr_per_sw = np.array(kcomp_df[electrode_column_name])
        desc = [f"kcomp@@{electd_arr}" for electd_arr in electd_arr_per_sw]
        new_format_kcomp = np.array([startTime_arr ,duration_arr,desc]).T ## onset (sec), duration,desc

        allsubsdata_perFile[id][kcomp_efdViewFormat_key] = new_format_kcomp
def save_eventsSepratelyForComparison_edfViewFormat(file_ids,events_types_to_compare,edfViewFormat_output_dir):
    # Save events (scoring and ss) per file, in a format suitable for EDF_viewer
    for id in file_ids:
        for event_type in events_types_to_compare:
            filename = f"{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_events"
            all_events_with_header = np.asarray([header],dtype=object)
            events_type_found = [ v for k, v in allsubsdata_perFile[id].items() if ((event_type in k) and (edfviewFormatSuffix in k))]
            if np.size(events_type_found) == 0: continue
            for events_found in events_type_found:
                    for event_found in events_found:
                            all_events_with_header = np.concatenate((all_events_with_header,[event_found]),dtype=object)
            np.savetxt(f"{edfViewFormat_output_dir}\\{event_type}_{filename}.txt", all_events_with_header, delimiter='\t',fmt='%s')
    
# Save events (scoring and ss) per file, in a format suitable for EDF_viewer
def save_eventsAllTogether_edfViewFormat(events_types_for_save,edfViewFormat_output_dir):
        for id in file_ids:
            filename = f"{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_events"
            all_events_with_header = np.asarray([header],dtype=object)
            for event_type in events_types_for_save:
                events_type_found = [ v for k, v in allsubsdata_perFile[id].items() if (event_type in k) and (edfviewFormatSuffix in k)]
                if np.size(events_type_found) == 0: continue
                for events_found in events_type_found:
                        for event_found in events_found:
                                all_events_with_header = np.concatenate((all_events_with_header,[event_found]),dtype=object)
            np.savetxt(f"{edfViewFormat_output_dir}\\allEvents_{filename}.txt", all_events_with_header, delimiter='\t',fmt='%s')

def preform_minmaxSD_comparison(events_types_to_compare,file_ids,minMax_sd_vers):
    all_comparisons = pd.DataFrame(columns=['id','eventType','minmax_ver','Positive','TruePos','FalsePos', 'hitRate','falseDiscoveryRate'])
    for id in file_ids:
        for ver in minMax_sd_vers:
            for event_type in events_types_to_compare:
                dir_auto_ = f"{edfViewFormat_events_output_dir}\\{ver}"
                manual_detection_filename = f"{edfViewFormat_events_manual_dir}\\sub{id.split('_')[0]}_sleep{id.split('_')[1]}_imported_eventDetectionChan_annotations.txt"
                auto_detection_filename = f"{dir_auto_}\\{event_type}_{id}_events.txt"
                ## get the array of before manual scanning
                if os.path.exists(auto_detection_filename) and os.path.exists(manual_detection_filename):
                    all_ss_auto = np.loadtxt(auto_detection_filename, delimiter="\t",dtype='object')
                    all_ss_auto = np.delete(all_ss_auto, np.where(all_ss_auto[:, 2] == "Annotation")[0], axis=0)
                    all_ss_auto[:,[0,1]] = [np.double(x) for x in all_ss_auto[:,[0,1]]]
                    ss_ind = np.array([],dtype=int)
                    for ind_i, desc in enumerate(all_ss_auto[:,2]):
                        if ("SS" in desc) or ("ss" in desc):
                            ss_ind = np.append(ss_ind, ind_i)
                    all_ss_auto = all_ss_auto[ss_ind,:]

                    ## get the array of after manual scanning
                    all_ss_manu = np.loadtxt(manual_detection_filename, delimiter="\t",dtype='object')
                    all_ss_manu = np.delete(all_ss_manu, np.where(all_ss_manu[:, 2] == "Annotation")[0], axis=0)
                    all_ss_manu[:,[0,1]] = [np.double(x) for x in all_ss_manu[:,[0,1]]]
                    ss_ind = np.array([],dtype=int)
                    for ind_i, desc in enumerate(all_ss_manu[:,2]):
                        if "SS" in desc:
                            ss_ind = np.append(ss_ind, ind_i)
                    all_ss_manu = all_ss_manu[ss_ind,:]

                    ## compare to find rate od TP and FP
                    TP = 0
                    FP = 0
                    for ss_auto in all_ss_auto:
                        found = False
                        for ss_manu in all_ss_manu:
                            if overlap([ss_auto[0],ss_auto[0]+ss_auto[1]],[ss_manu[0],ss_manu[0]+ss_manu[1]]):
                                TP +=1
                                found = True
                                break
                        if found == False:
                            FP +=1

                    # FN = 0
                    # for ss_manu in all_ss_manu:
                    #     found = False
                    #     for ss_auto in all_ss_auto:
                    #         if overlap([ss_auto[0],ss_auto[0]+ss_auto[1]],[ss_manu[0],ss_manu[0]+ss_manu[1]]):
                    #             found = True
                    #             break
                    #     if found == False:
                    #         FN +=1

                    Positive = np.shape(all_ss_manu)[0]   
                    hitRate = TP/Positive
                    #missRate = FN/Positive
                    falseDiscoveryRate = FP/(FP+TP)
                    
                    comparison = [id,event_type, ver, Positive, TP, FP,hitRate, falseDiscoveryRate]
                    all_comparisons.loc[len(all_comparisons)] = comparison

    all_comparisons = all_comparisons.sort_values('minmax_ver')
    all_comparisons = all_comparisons.sort_values('id')
    return all_comparisons


In [89]:
## RUN THIS TO GET COMPARISONS OF SPINDLES MINMANX_SD OPTIONS
run_comparison_of_spindle_minMax = False
if(run_comparison_of_spindle_minMax):
    testType = 0
    if(testType==1):
        file_ids = ['38_2']
        minMax_sd_vers = [[4,10,1]]
        electrodes_names_eventDetect = ['Fp2','F3']
    elif(testType==2):
        file_ids = ['32_2','38_2']
        minMax_sd_vers = [[3,9,1],[4,10,1]]
        electrodes_names_eventDetect = ['Fp2','F3']
    elif(testType==3):
        file_ids = ['32_2','38_2']
        minMax_sd_vers = [[3,9,1],[4,10,1]]
        electrodes_names_eventDetect = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2']
    else:
        file_ids = ['32_2','38_2']
        electrodes_names_eventDetect = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2']
        minMax_sd_vers = [[3,8,1],[4,8,1],[5,8,1],[3,9,1],[4,9,1],[5,9,1],[3,10,1],[4,10,1],[5,10,1]]
        #file_ids = allsubsdata_perFile.keys()

    with open(import_path, "rb") as file:
        [allsubsdata_perFile, configu] = pickle.load(file)
    fig_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}"
    if not os.path.exists(fig_output_dir): os.mkdir(fig_output_dir)

    edfViewFormat_events_manual_dir = f'{fig_output_dir}\\EDFViewFormat_events'
    edfViewFormat_events_output_dir = f'{fig_output_dir}\\EDFViewFormat_events\\minmax_sd_tests'
    if os.path.exists(edfViewFormat_events_output_dir):
        shutil.rmtree(edfViewFormat_events_output_dir)
    os.mkdir(edfViewFormat_events_output_dir)

    fig_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}"
    if not os.path.exists(fig_output_dir): os.mkdir(fig_output_dir)

    for minMax_sd_ver in minMax_sd_vers:
        with open(import_path, "rb") as file: [allsubsdata_perFile, configu] = pickle.load(file)
        
        all_electodes_ss_key = 'SS_AN'
        minmax_sd_name = detect_ss_AndrillonNir(file_ids,minMax_sd_ver,all_electodes_ss_key,electrodes_names_eventDetect)
        print(minmax_sd_name)

        after_sleepStage_exclution_key = f"{all_electodes_ss_key}_n2n3"
        filter_sleep_stages(file_ids,all_electodes_ss_key,after_sleepStage_exclution_key, [2.0,3.0]) 

        sw_key_to_use = after_sleepStage_exclution_key
        multiElectdPerSS_text = "multiElectd"
        singleElectdPerSS_text = "singleElectd"
        multiElectdPerSS_key = f"{sw_key_to_use}_{multiElectdPerSS_text}"
        singleElectdPerSS_key = f"{sw_key_to_use}_{singleElectdPerSS_text}"
        group_spindles(sw_key_to_use,multiElectdPerSS_key, singleElectdPerSS_key)

        edfviewFormatSuffix = 'efdViewFormat'
        SS_multiElectdPerSS_efdViewFormat_key = f'{multiElectdPerSS_key}_{edfviewFormatSuffix}'
        SS_singleElectdPerSS_efdViewFormat_key = f'{singleElectdPerSS_key}_{edfviewFormatSuffix}'
        add_edfViewFormat_ss(file_ids,multiElectdPerSS_key,SS_multiElectdPerSS_efdViewFormat_key)
        add_edfViewFormat_ss(file_ids,singleElectdPerSS_key,SS_singleElectdPerSS_efdViewFormat_key)

        events_types_to_compare = [multiElectdPerSS_text,singleElectdPerSS_text]    
        edfViewFormat_eventsTest_output_dir = f"{edfViewFormat_events_output_dir}\\{minMax_sd_ver}"
        if os.path.exists(edfViewFormat_eventsTest_output_dir):  shutil.rmtree(edfViewFormat_eventsTest_output_dir)
        os.mkdir(edfViewFormat_eventsTest_output_dir)
        save_eventsSepratelyForComparison_edfViewFormat(file_ids,events_types_to_compare,edfViewFormat_eventsTest_output_dir)

        scoring_edfViewFormat_key = f'scoring_{edfviewFormatSuffix}'
        add_edfViewFormat_scoring(scoring_edfViewFormat_key)

        events_types_to_save = ['scoring', multiElectdPerSS_text,singleElectdPerSS_text]    
        save_eventsAllTogether_edfViewFormat(events_types_to_save,edfViewFormat_eventsTest_output_dir)

In [90]:
run_comparison_of_spindle_minMax = False
if(run_comparison_of_spindle_minMax):
    events_types_to_compare = [multiElectdPerSS_text,singleElectdPerSS_text]
    all_comp = preform_minmaxSD_comparison(events_types_to_compare,file_ids,minMax_sd_vers)
    # all_comp.to_csv('ss_comparisons.csv',index=False)
    display(all_comp)

In [91]:
## RUN this to get scoring events files only
run_scoring_eventFiles = False
if(run_scoring_eventFiles):
    testType = 1
    if(testType==1):
        with open(import_path, "rb") as file: [allsubsdata_perFile, configu] = pickle.load(file)
        file_ids = allsubsdata_perFile.keys()
        electrodes_names_eventDetect = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2']


    fig_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}"
    if not os.path.exists(fig_output_dir): os.mkdir(fig_output_dir)

    edfViewFormat_events_manual_dir = f'{fig_output_dir}\\EDFViewFormat_events'
    edfViewFormat_events_output_dir = f'{fig_output_dir}\\EDFViewFormat_events\\only_scoring'
    if os.path.exists(edfViewFormat_events_output_dir):
        shutil.rmtree(edfViewFormat_events_output_dir)
    os.mkdir(edfViewFormat_events_output_dir)

    all_electodes_sw_key = 'SW_AN'

    edfviewFormatSuffix = 'efdViewFormat'

    scoring_edfViewFormat_key = f'scoring_{edfviewFormatSuffix}'
    add_edfViewFormat_scoring(scoring_edfViewFormat_key)

    events_types_to_save = ['scoring']    
    save_eventsAllTogether_edfViewFormat(events_types_to_save,edfViewFormat_events_output_dir)  

In [139]:
## RUN THIS TO GET AN_SlowWaves events
run_comparison_of_sw_minMax = True
if(run_comparison_of_sw_minMax):
    testType = 1
    if(testType==1):
        file_ids = ['38_2']
        electrodes_names_eventDetect = ['Fp2','F3']
    elif(testType==2):
        file_ids = ['32_2','38_2']
        electrodes_names_eventDetect = ['Fp2','F3']
    elif(testType==3):
        file_ids = ['32_2','38_2']
        electrodes_names_eventDetect = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2']
    else:
        file_ids = ['32_2','38_2']
        electrodes_names_eventDetect = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2']

    with open(import_path, "rb") as file:  [allsubsdata_perFile, configu] = pickle.load(file)
    fig_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}"
    if not os.path.exists(fig_output_dir): os.mkdir(fig_output_dir)

    edfViewFormat_events_manual_dir = f'{fig_output_dir}\\EDFViewFormat_events'
    edfViewFormat_events_output_dir = f'{fig_output_dir}\\EDFViewFormat_events\\sw_AN_tests'
    if os.path.exists(edfViewFormat_events_output_dir):
        shutil.rmtree(edfViewFormat_events_output_dir)
    os.mkdir(edfViewFormat_events_output_dir)

    
    all_electodes_sw_key = 'SW_AN'
    res = detect_sw_AndrillonNir(file_ids,all_electodes_sw_key,electrodes_names_eventDetect)

    after_sleepStage_exclution_key = f"{all_electodes_sw_key}_n2n3"
    filter_sleep_stages(file_ids,all_electodes_sw_key,after_sleepStage_exclution_key, [2.0,3.0]) 

    sw_key_to_use = after_sleepStage_exclution_key
    sw_key_after_SWExclusion = f"{after_sleepStage_exclution_key}_swExclusion"
    filter_waveCriteria(file_ids,sw_key_to_use,sw_key_after_SWExclusion)

    edfviewFormatSuffix = 'efdViewFormat'
    SW_multiElectdPerSW_efdViewFormat_key = f'{sw_key_after_SWExclusion}_{edfviewFormatSuffix}'
    add_edfViewFormat_sw(file_ids,sw_key_after_SWExclusion,SW_multiElectdPerSW_efdViewFormat_key)

    # events_types_to_compare = [multiElectdPerSW_text,singleElectdPerSW_text]    
    # edfViewFormat_eventsTest_output_dir = f"{edfViewFormat_events_output_dir}\\{minMax_sd_ver}"
    # if os.path.exists(edfViewFormat_eventsTest_output_dir):  shutil.rmtree(edfViewFormat_eventsTest_output_dir)
    # os.mkdir(edfViewFormat_eventsTest_output_dir)
    # save_eventsSepratelyForComparison_edfViewFormat(file_ids,events_types_to_compare,edfViewFormat_eventsTest_output_dir)

    scoring_edfViewFormat_key = f'scoring_{edfviewFormatSuffix}'
    add_edfViewFormat_scoring(scoring_edfViewFormat_key)

    events_types_to_save = ['scoring', SW_multiElectdPerSW_efdViewFormat_key]    
    save_eventsAllTogether_edfViewFormat(events_types_to_save,edfViewFormat_events_output_dir)

08-מרץ-23 00:12:22 | WARNING | Hypnogram is LONGER than data by 6.99 seconds. Cropping hypnogram to match data.size.


sub:38_2, before exclude sleep stages:20237, after:11281
sub:38_2, before wave criteria sleep stages:11281, after:2324


In [130]:
def filter_kcompCriteria(file_ids,event_key_to_use,afterkcompCriteria_exclution_key):
            # % Result Matrix
        # %1:  wave beginning (sample)
        # %2:  wave end (sample)
        # %3:  wave middle point (sample)
        # %4:  wave negative half-way (sample)
        # %5:  period in seconds
        # %6:  positive amplitude peak
        # %7:  positive amplitude peak position (sample)
        # %8:  negative amplitude peak
        # %9:  negative amplitude peak position (sample)
        # %10: peak-to-peak amplitude
        # %11: 1st pos peak amplitude
        # %12: 1st pos peak amplitude position (sample)
        # %13: Last pos peak amplitude
        # %14: Last pos peak amplitude position (sample)
        # %15: 1st neg peak amplitude
        # %16: 1st neg peak amplitude position (sample)
        # %17: mean wave amplitude
        # %18: number of positive peaks
        # %19: wave negative half-way period
        # %20: 1st peak to last peak period
        # %21: determines instantaneous positive 1st segement slope on smoothed signal
        # %22: determines maximal negative slope for 2nd segement
        # %23: stage (if scored data)
    for id in file_ids:
        if event_key_to_use not in allsubsdata_perFile[id]: continue
        event_allElectrodes = allsubsdata_perFile[id][event_key_to_use]
        event_filtered_kcompCrit = event_allElectrodes[       (event_allElectrodes['n1'] < 45) 
                                                        &((event_allElectrodes['currentStage'] == 2.0) | (event_allElectrodes['currentStage'] == 3.0))
                                                       ]
        
    
        if len(event_filtered_kcompCrit) == 0:  print(f"sub:{id} - no kcomp with criteria")
        else: print(f"sub:{id}, before wave criteria sleep stages:{len(event_allElectrodes)}, after:{len(event_filtered_kcompCrit)}")
        
        allsubsdata_perFile[id][afterkcompCriteria_exclution_key] = event_filtered_kcompCrit

## RUN THIS TO GET AN_kcomplex events
run_comparison_of_kcomp_minMax = False
if(run_comparison_of_kcomp_minMax):
    testType = 1
    if(testType==1):
        file_ids = ['38_2']
        electrodes_names_eventDetect = ['Fp2']
    elif(testType==2):
        file_ids = ['32_2','38_2']
        electrodes_names_eventDetect = ['Fp2','F3']
    else:
        file_ids = ['32_2','38_2']
        electrodes_names_eventDetect = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2']

    with open(import_path, "rb") as file:  [allsubsdata_perFile, configu] = pickle.load(file)
    fig_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}"
    if not os.path.exists(fig_output_dir): os.mkdir(fig_output_dir)

    edfViewFormat_events_manual_dir = f'{fig_output_dir}\\EDFViewFormat_events'
    edfViewFormat_events_output_dir = f'{fig_output_dir}\\EDFViewFormat_events\\kcomp_tests_AN'
    if os.path.exists(edfViewFormat_events_output_dir):
        shutil.rmtree(edfViewFormat_events_output_dir)
    os.mkdir(edfViewFormat_events_output_dir)

    all_electodes_kcomp_key = 'KComp'
    detect_kComp_AndrillonNir(file_ids,all_electodes_kcomp_key,electrodes_names_eventDetect)

    after_criteria_key = f"{all_electodes_kcomp_key}_criteria"
    filter_kcompCriteria(file_ids,all_electodes_kcomp_key,after_criteria_key)

    edfviewFormatSuffix = 'efdViewFormat'
    kcomp_efdViewFormat_key = f'{after_criteria_key}_{edfviewFormatSuffix}'
    add_edfViewFormat_kcomp(file_ids,after_criteria_key,kcomp_efdViewFormat_key)

    scoring_edfViewFormat_key = f'scoring_{edfviewFormatSuffix}'
    add_edfViewFormat_scoring(scoring_edfViewFormat_key)

    events_types_to_save = ['scoring', kcomp_efdViewFormat_key]    
    save_eventsAllTogether_edfViewFormat(events_types_to_save,edfViewFormat_events_output_dir)

In [126]:
event_allElectrodes = allsubsdata_perFile['38_2']['KComp_criteria']
event_filtered_kcompCrit = pd.DataFrame([], columns=event_allElectrodes.columns)

neg_events_filtered = event_allElectrodes[       (event_allElectrodes['n1'] < 40) &
                                                ((event_allElectrodes['currentStage'] == 2.0) | (event_allElectrodes['currentStage'] == 3.0))
                                                ]
neg_events_filtered.reset_index(drop=True, inplace=True)

exclude_indxs = np.zeros([], dtype=int)
for index, row in neg_events_filtered.iterrows():
    if index == 0: continue
    #print(event_filtered_neg.iloc[index-1]['wavend'] +1)
    if neg_events_filtered.iloc[index-1]['n1x'] +1.5 >=neg_events_filtered.iloc[index]['n1x']: # filtered minimas that are seperared by at leaset 1.5s
        exclude_indxs = np.append(exclude_indxs, index)

ex = neg_events_filtered.drop(exclude_indxs, axis=0)
ex.reset_index(drop=True, inplace=True)

print(np.size(allsubsdata_perFile['38_2']['KComp_criteria'],0))
print(allsubsdata_perFile['38_2']['KComp_criteria_efdViewFormat'])
display(allsubsdata_perFile['38_2']['KComp_criteria'])
a= allsubsdata_perFile['38_2']['KComp_criteria']

7838
[['0.94' '0.21' 'kcomp@@Fp2']
 ['1.15' '1.84' 'kcomp@@Fp2']
 ['4.32' '0.42' 'kcomp@@Fp2']
 ...
 ['9177.81' '0.42' 'kcomp@@Fp2']
 ['9178.78' '0.21' 'kcomp@@Fp2']
 ['9178.99' '1.03' 'kcomp@@Fp2']]


,wavest,wavend,mdpt,poszx,period,abs(b),bx,c,cx,maxb2c,...,p1,p1x,meanAmp,nump,nperiod/SR,p2p,mxdn,mxup,currentStage,electrode
1,0.94,1.15,1.00,1.06,0.21,6.971936,1.02,5.045137,1.14,12.017073,...,5.045137,1.14,6.971936,1.0,0.024,0.040,802.804395,687.519515,3.0,Fp2
2,1.15,2.99,1.56,1.96,1.84,61.083939,1.66,79.951886,2.26,141.035826,...,79.951886,2.26,45.178569,2.0,0.162,0.224,2307.625153,2034.929110,3.0,Fp2
5,4.32,4.74,4.37,4.42,0.42,14.050622,4.40,53.656340,4.57,67.706962,...,53.656340,4.57,14.050622,1.0,0.020,0.074,2520.911287,1531.321931,3.0,Fp2
7,5.64,5.82,5.68,5.72,0.18,1.723316,5.71,6.597173,5.80,8.320489,...,6.597173,5.80,1.723316,1.0,0.016,0.066,406.329021,145.938325,3.0,Fp2
8,5.82,6.31,5.89,5.96,0.49,16.218847,5.93,67.255603,6.23,83.474450,...,48.738177,6.09,16.218847,1.0,0.028,0.086,1415.443215,1382.480299,3.0,Fp2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16788,9175.96,9176.98,9176.01,9176.06,1.02,6.695662,9176.03,29.953387,9176.39,36.649049,...,2.223227,9176.12,6.695662,1.0,0.020,0.128,962.165160,668.175160,2.0,Fp2
16789,9176.98,9177.81,9177.26,9177.53,0.83,41.230901,9177.09,15.089083,9177.75,56.319984,...,12.683360,9177.63,38.672957,2.0,0.110,0.272,3098.875626,2274.378160,2.0,Fp2
16790,9177.81,9178.23,9177.90,9177.98,0.42,22.681655,9177.91,12.188371,9178.08,34.870026,...,12.188371,9178.08,22.681655,1.0,0.034,0.066,1989.869484,1689.830890,2.0,Fp2
16792,9178.78,9178.99,9178.86,9178.93,0.21,27.942224,9178.86,1.871429,9178.98,29.813653,...,1.871429,9178.98,27.942224,1.0,0.030,0.046,3643.961421,2215.146140,2.0,Fp2


In [110]:
## RUN Sridhar_kcomplex events

# Sample rate and desired cutoff frequencies (in Hz).
wanted_freq = 100
lowcut = 0.25
highcut = 6.0

number_of_sec_in_epoch = 4
distance_in_sec = 1.5
distance_in_samples = wanted_freq * distance_in_sec
min_negPos_gap = 0.2 # in sec
max_negPos_gap = 1.5 # in sec
range_for_pos_peak = 1 #in sec

kcomp_df_columns = ['pos_i','pos_amp','neg_i','neg_amp',electrode_column_name]
kcomp_df =  pd.DataFrame([], columns = kcomp_df_columns)

def detect_kComp_Srid(file_ids,output_key,electrodes_names):
    for id in file_ids:
        kcomp_df =  pd.DataFrame([], columns = kcomp_df_columns)

        datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
        datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
        scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'])
        
        for electrode_name in electrodes_names:
            curr_electrode_num = np.where(configu['electrodes_names'] == electrode_name)[0][0]
            data = allsubsdata_perFile[id]['data'][curr_electrode_num] ## shape (time)
            z = wanted_freq / configu['sample_freq']
            resampled = signal.resample(data, int(np.size(data)*z)) ## downsample to 100hz
            filtered = butter_bandpass_filter(resampled, lowcut, highcut, wanted_freq, order=6) ## oreder can be changed. see: https://stackoverflow.com/questions/12093594/how-to-implement-band-pass-butterworth-filter-with-scipy-signal-butter

            for epoch_i in np.arange(np.size(filtered)/wanted_freq/number_of_sec_in_epoch,dtype=int):
                if epoch_i == 0: continue
                curr_epoch_orig_i = int(epoch_i*wanted_freq*number_of_sec_in_epoch/z)
                if (scoring_upsampled[curr_epoch_orig_i] ==2 or scoring_upsampled[curr_epoch_orig_i] ==3):
                    curr_start_time = (epoch_i-1)*wanted_freq*number_of_sec_in_epoch
                    curr_end_time = epoch_i*wanted_freq*number_of_sec_in_epoch
                    epoch_data = filtered[int(curr_start_time) : int(curr_end_time)]
                    scaled_epoch_data = max(epoch_data) - epoch_data; #scale the signal by the maximum value.
                    
                    curr_minimas_inds, _ = find_peaks(-epoch_data, height=40, distance=distance_in_samples)

                    for minima_ind in curr_minimas_inds:
                        if epoch_data[minima_ind] < -40:
                            start_range = max(0, minima_ind-(range_for_pos_peak *wanted_freq))
                            end_range = min((number_of_sec_in_epoch *wanted_freq)-1, minima_ind+(range_for_pos_peak *wanted_freq))               
                            
                            curr_maximas_inds, _ = find_peaks(epoch_data[start_range:end_range], height=0)
                            if np.size(curr_minimas_inds) ==0 or np.size(epoch_data[curr_maximas_inds])==0: continue
                            curr_maximas_inds = curr_maximas_inds+start_range
                            
                            max_ind = curr_maximas_inds[np.argmax(epoch_data[curr_maximas_inds])] 
                            
                            if np.abs(max_ind - minima_ind) < max_negPos_gap*wanted_freq and np.abs(max_ind - minima_ind) > min_negPos_gap*wanted_freq:
                                ## TODO: check if the negative peak is below ~45milivolt
                                if epoch_data[max_ind] >= 2* epoch_data[minima_ind]:
                                    if np.abs(epoch_data[max_ind] - epoch_data[minima_ind]) > 100:
                                        new_row = {'pos_i':(curr_start_time+max_ind)/z, 
                                                            'pos_amp':epoch_data[max_ind], 
                                                            'neg_i':(curr_start_time+minima_ind)/z, 
                                                            'neg_amp':epoch_data[minima_ind],
                                                            electrode_column_name:electrode_name}
                                        #kcomp_df = kcomp_df.append(new_row, ignore_index=True)
                                        new_row_df = pd.DataFrame([new_row], columns = kcomp_df_columns)
                                        kcomp_df = pd.concat([kcomp_df,new_row_df])

        if not kcomp_df.empty:
            allsubsdata_perFile[id][output_key]  = kcomp_df
def add_edfViewFormat_kcomp_Srid(file_ids,event_key_for_save, kcomp_efdViewFormat_key):
    for id in file_ids:
        if event_key_for_save not in allsubsdata_perFile[id]:
                #print(f"no {event_key_for_save} for sub {id}")
                continue
        kcomp_df = allsubsdata_perFile[id][event_key_for_save]

        startTime_arr = np.array(kcomp_df['neg_i'])
        endTime_arr = np.array(kcomp_df['pos_i']) 
        duration_arr = np.round((endTime_arr - startTime_arr)/configu['sample_freq'],2)
        electd_arr_per_sw = np.array(kcomp_df[electrode_column_name])
        desc = [f"kcomp@@{electd_arr}" for electd_arr in electd_arr_per_sw]
        new_format_kcomp = np.array([startTime_arr /configu['sample_freq'],duration_arr,desc]).T ## onset (sec), duration,desc

        allsubsdata_perFile[id][kcomp_efdViewFormat_key] = new_format_kcomp

## RUN THIS TO GET Sridhar_kcomplex events
run_kcomp_srid = False
if(run_kcomp_srid):
    testType = 1
    if(testType==1):
        file_ids = ['38_2']
        electrodes_names_eventDetect = ['Fp2']
    elif(testType==2):
        file_ids = ['32_2','38_2']
        electrodes_names_eventDetect = ['Fp2','F3']
    else:
        file_ids = ['32_2','38_2']
        electrodes_names_eventDetect = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2']

    with open(import_path, "rb") as file:  [allsubsdata_perFile, configu] = pickle.load(file)
    fig_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}"
    if not os.path.exists(fig_output_dir): os.mkdir(fig_output_dir)

    edfViewFormat_events_manual_dir = f'{fig_output_dir}\\EDFViewFormat_events'
    edfViewFormat_events_output_dir = f'{fig_output_dir}\\EDFViewFormat_events\\kcomp_tests_Sridhar'
    if os.path.exists(edfViewFormat_events_output_dir):
        shutil.rmtree(edfViewFormat_events_output_dir)
    os.mkdir(edfViewFormat_events_output_dir)

    all_electodes_kcompSrid_key = 'KComp_Srid'
    res = detect_kComp_Srid(file_ids,all_electodes_kcompSrid_key,electrodes_names_eventDetect)

    edfviewFormatSuffix = 'efdViewFormat'
    kcomp_efdViewFormat_key = f'{all_electodes_kcompSrid_key}_{edfviewFormatSuffix}'
    add_edfViewFormat_kcomp_Srid(file_ids,all_electodes_kcompSrid_key,kcomp_efdViewFormat_key)

    scoring_edfViewFormat_key = f'scoring_{edfviewFormatSuffix}'
    add_edfViewFormat_scoring(scoring_edfViewFormat_key)

    events_types_to_save = ['scoring', kcomp_efdViewFormat_key]    
    save_eventsAllTogether_edfViewFormat(events_types_to_save,edfViewFormat_events_output_dir)

07-מרץ-23 16:35:55 | WARNING | Hypnogram is LONGER than data by 6.99 seconds. Cropping hypnogram to match data.size.


In [35]:
## RUN ShaYKM_kcomplex events

# Sample rate and desired cutoff frequencies (in Hz).
wanted_freq = 100

distance_in_sec = 1.5
distance_in_sw = wanted_freq * distance_in_sec
min_negPos_gap = 0.2 # in sec
max_negPos_gap = 1.5 # in sec
range_for_pos_peak = 1 #in sec

kcomps_dur_min = 0.5 # sec

sw_min_width = 0.3 # sec

kcomp_df_columns = ['pos_i','pos_amp','neg_i','neg_amp',electrode_column_name]
kcomp_df =  pd.DataFrame([], columns = kcomp_df_columns)

def detect_kComp_shaYKM(file_ids,output_key,electrodes_names):
    for id in file_ids:
        kcomp_df =  pd.DataFrame([], columns = kcomp_df_columns)

        datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
        datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
        scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'])
        
        for electrode_name in electrodes_names:
            curr_electrode_num = np.where(configu['electrodes_names'] == electrode_name)[0][0]
            data = allsubsdata_perFile[id]['data'][curr_electrode_num] ## shape (time)
            z = wanted_freq / configu['sample_freq']
            resampled = signal.resample(data, int(np.size(data)*z)) ## downsample to 100hz

            smoothing_window = 3
            smoothen  = scipy.signal.medfilt(resampled, smoothing_window)
            lowcut = 0.25
            highcut = 3.0
            filer_order = 3
            filtered_sw = butter_bandpass_filter(smoothen, lowcut, highcut, wanted_freq, order=filer_order) ## oreder can be changed. see: https://stackoverflow.com/questions/12093594/how-to-implement-band-pass-butterworth-filter-with-scipy-signal-butter
            
            smoothing_window = 3
            smoothen  = scipy.signal.medfilt(resampled, smoothing_window)
            lowcut = 0.5
            highcut = 6.0
            filer_order = 3
            filtered_transiantMinima = butter_bandpass_filter(smoothen, lowcut, highcut, wanted_freq, order=filer_order)
            
            ## find all standalone minimas
            curr_maximas_inds, _ = find_peaks(filtered_sw, threshold=0,width=sw_min_width*wanted_freq, distance=distance_in_sw)
            for maxima_ind in curr_maximas_inds:
                if (scoring_upsampled[int(maxima_ind/z)] ==2):
                    # plt.plot(resampled[max(minima_ind-200,0):min(minima_ind+200,np.size(resampled)-1)])
                    # plt.plot(filtered_sw[max(minima_ind-200,0):min(minima_ind+200,np.size(resampled)-1)])
                    # plt.plot(filtered_notSW[max(minima_ind-200,0):min(minima_ind+200,np.size(resampled)-1)])
                    # plt.show()
                    maxima = filtered_sw[maxima_ind]

                    range_for_neg_peak = 1 #in sec
                    minima_scope = filtered_transiantMinima[maxima_ind - int(range_for_neg_peak*wanted_freq) : maxima_ind]
                    distance_in_preSW = 1
                    curr_minimas_inds, _ = find_peaks(-minima_scope, threshold=0,distance=distance_in_preSW)

                    if np.size(curr_minimas_inds) ==0 : continue

                    min_minima_ind_inScope = curr_minimas_inds[np.argmin(minima_scope[curr_minimas_inds])]
                    minima = minima_scope[min_minima_ind_inScope]
                    minima_ind = min_minima_ind_inScope + (maxima_ind - int(range_for_neg_peak*wanted_freq))

                    if ((maxima_ind - minima_ind) > kcomps_dur_min *wanted_freq )and (minima < maxima-100) and (minima<-45) and (maxima > 0.5*abs(minima)): ## found a k-comp!
                        new_row =           {'pos_i':maxima_ind/z, 
                                            'pos_amp':maxima, 
                                            'neg_i':minima_ind/z, 
                                            'neg_amp':minima,
                                            electrode_column_name:electrode_name}
                        new_row_df = pd.DataFrame([new_row], columns = kcomp_df_columns)
                        kcomp_df = pd.concat([kcomp_df,new_row_df])
            
        if not kcomp_df.empty:
            allsubsdata_perFile[id][output_key]  = kcomp_df
        else: print(f'no events: {output_key}')
            
def add_edfViewFormat_kcomp_shaYKM(file_ids,event_key_for_save, kcomp_efdViewFormat_key):
    for id in file_ids:
        if event_key_for_save not in allsubsdata_perFile[id]:
                #print(f"no {event_key_for_save} for sub {id}")
                continue
        kcomp_df = allsubsdata_perFile[id][event_key_for_save]

        startTime_arr = np.array(kcomp_df['neg_i'])
        endTime_arr = np.array(kcomp_df['pos_i']) 
        duration_arr = np.round((endTime_arr - startTime_arr)/configu['sample_freq'],2)
        electd_arr_per_sw = np.array(kcomp_df[electrode_column_name])
        desc = [f"kcomp@@{electd_arr}" for electd_arr in electd_arr_per_sw]
        new_format_kcomp = np.array([startTime_arr /configu['sample_freq'],duration_arr,desc]).T ## onset (sec), duration,desc

        allsubsdata_perFile[id][kcomp_efdViewFormat_key] = new_format_kcomp


run_kcomp_shaYKM = True
if(run_kcomp_shaYKM):
    testType = 2
    if(testType==1):
        file_ids = ['32_2']
        electrodes_names_eventDetect = ['F4']
    elif(testType==2):
        file_ids = ['32_2','38_2']
        electrodes_names_eventDetect = ['F4']
    else:
        file_ids = ['32_2','38_2']
        electrodes_names_eventDetect = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2']

    with open(import_path, "rb") as file:  [allsubsdata_perFile, configu] = pickle.load(file)
    fig_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}"
    if not os.path.exists(fig_output_dir): os.mkdir(fig_output_dir)

    edfViewFormat_events_manual_dir = f'{fig_output_dir}\\EDFViewFormat_events'
    edfViewFormat_events_output_dir = f'{fig_output_dir}\\EDFViewFormat_events\\kcomp_tests_ShaYKM'
    if os.path.exists(edfViewFormat_events_output_dir):
        shutil.rmtree(edfViewFormat_events_output_dir)
    os.mkdir(edfViewFormat_events_output_dir)

    all_electodes_kcompSrid_key = 'KComp_shaYKM'
    res = detect_kComp_shaYKM(file_ids,all_electodes_kcompSrid_key,electrodes_names_eventDetect)

    edfviewFormatSuffix = 'efdViewFormat'
    kcomp_efdViewFormat_key = f'{all_electodes_kcompSrid_key}_{edfviewFormatSuffix}'
    add_edfViewFormat_kcomp_shaYKM(file_ids,all_electodes_kcompSrid_key,kcomp_efdViewFormat_key)

    scoring_edfViewFormat_key = f'scoring_{edfviewFormatSuffix}'
    add_edfViewFormat_scoring(scoring_edfViewFormat_key)

    events_types_to_save = ['scoring', kcomp_efdViewFormat_key]    
    save_eventsAllTogether_edfViewFormat(events_types_to_save,edfViewFormat_events_output_dir)

allsubsdata_perFile['32_2'][all_electodes_kcompSrid_key].reset_index()

08-מרץ-23 03:43:18 | WARNING | Hypnogram is LONGER than data by 3.59 seconds. Cropping hypnogram to match data.size.
08-מרץ-23 03:43:19 | WARNING | Hypnogram is LONGER than data by 6.99 seconds. Cropping hypnogram to match data.size.


,index,pos_i,pos_amp,neg_i,neg_amp,electrode
0,0,301625.0,126.403798,301375.0,-118.909419,F4
1,0,306680.0,72.792779,306265.0,-68.405855,F4
2,0,311660.0,198.979489,311370.0,-97.359740,F4
3,0,325185.0,112.552360,324930.0,-71.395604,F4
4,0,336840.0,121.246554,336590.0,-101.798268,F4
...,...,...,...,...,...,...
131,0,4428520.0,146.594144,4428215.0,-96.804175,F4
132,0,4430895.0,96.296364,4430730.0,-106.690510,F4
133,0,4433405.0,61.609141,4433215.0,-119.192816,F4
134,0,4495285.0,84.974502,4495010.0,-104.821969,F4
